# Load Package

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
cur_path = "/content/drive/MyDrive/Bigdataproject/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Bigdataproject


In [3]:
!pip install pyspark
!pip install pyspark[sql]
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=9a10dc860f467b3e9113c9c3674a7fcff82c9304d17198b891af10bb7f6083f3
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-de

In [4]:
import pyspark as spark
import time
from operator import add
from pyspark.sql import SparkSession

# Step 0: Read Data

## Set spark session

In [5]:
# set spark session
ss = (SparkSession
  .builder
  .master("local[5]")
  .appName("fqproject")
  .getOrCreate())


## Set spark context

In [6]:
# set spark session
sc = (spark
  .SparkContext
  .getOrCreate(spark
    .SparkConf()
    .setAppName("fqproject")
    .setMaster('local[*]')
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '4G')
    .set('spark.driver.maxResultSize', '4G')))

## Read reads.fq

In [7]:
# read reads.fq and extract the second line of every four lines.
reads = (sc
  .textFile(cur_path + "data/reads.fq")
  .zipWithIndex()
  .filter(lambda x: (x[1]+1)%2==0 and (x[1]+1)%4!=0)
  .map(lambda x: x[0]))

In [8]:
reads.take(10)

['TCCTTACTGGTTTTGCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATC',
 'GGTTTTTCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTGGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATA',
 'AGGTAACTTATAGAGTACTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTGATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTG',
 'CACTTCCCTTCTACTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATG',
 'CCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCCCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTTTT',
 'ATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCGTGCCTATTAGATTCATT',
 'TCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATT',
 'TATCCATATGGCATAATCAAAGAATCAATTGTTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATTTTC',
 'TCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGATTATAAAAAAATTTTATTTTATTTTCACTTATTTCTTCTCCA',
 'TGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTACTAGATTCATTTAG

## Read reference.fa

In [9]:
# read reference
reference = (ss
  .read
  .csv(
      cur_path + "data/reference_chr21_20000000_20050000.fa",
      inferSchema=True,
      header=True))

In [10]:
reference.show(10)

+--------+--------+--------------------+
|   start|     end|            sequence|
+--------+--------+--------------------+
|20000000|20000100|CCCTTCTCCTATCCCTT...|
|20000100|20000200|TAGATTCATTTAGAATA...|
|20000200|20000300|TTCTTCTTCCTGAAGAA...|
|20000300|20000400|ACTTTTCAAGGATAGTT...|
|20000400|20000500|CTGACAGGACTTCTGCC...|
|20000500|20000600|TTTCCTTTTTTTTTTCT...|
|20000600|20000700|ATTATAAAAAGGGAGGG...|
|20000700|20000800|TTCTTTTCTTTTTCTCC...|
|20000800|20000900|ATAAATTTCTGCTTGAA...|
|20000900|20001000|TTCGTTAGTGTTTTTTA...|
+--------+--------+--------------------+
only showing top 10 rows



# Step 1: Cut reads into kmers = 15

## Define K-mers

In [11]:
# input a string s and cut k-mers
def extract_kmers(s: str, k: int = 15) -> list:
    return [s[i:i+k] for i in range(len(s)-k+1)]

## Map K-mers to reads

In [12]:
# K-mers of reads
reads_kmers = (reads
  .map(extract_kmers))

In [13]:
reads_kmers.take(2)

[['TCCTTACTGGTTTTG',
  'CCTTACTGGTTTTGC',
  'CTTACTGGTTTTGCA',
  'TTACTGGTTTTGCAG',
  'TACTGGTTTTGCAGG',
  'ACTGGTTTTGCAGGT',
  'CTGGTTTTGCAGGTA',
  'TGGTTTTGCAGGTAA',
  'GGTTTTGCAGGTAAC',
  'GTTTTGCAGGTAACT',
  'TTTTGCAGGTAACTT',
  'TTTGCAGGTAACTTA',
  'TTGCAGGTAACTTAT',
  'TGCAGGTAACTTATA',
  'GCAGGTAACTTATAG',
  'CAGGTAACTTATAGA',
  'AGGTAACTTATAGAG',
  'GGTAACTTATAGAGT',
  'GTAACTTATAGAGTA',
  'TAACTTATAGAGTAT',
  'AACTTATAGAGTATT',
  'ACTTATAGAGTATTT',
  'CTTATAGAGTATTTC',
  'TTATAGAGTATTTCC',
  'TATAGAGTATTTCCA',
  'ATAGAGTATTTCCAC',
  'TAGAGTATTTCCACT',
  'AGAGTATTTCCACTT',
  'GAGTATTTCCACTTC',
  'AGTATTTCCACTTCC',
  'GTATTTCCACTTCCC',
  'TATTTCCACTTCCCT',
  'ATTTCCACTTCCCTT',
  'TTTCCACTTCCCTTC',
  'TTCCACTTCCCTTCT',
  'TCCACTTCCCTTCTC',
  'CCACTTCCCTTCTCC',
  'CACTTCCCTTCTCCT',
  'ACTTCCCTTCTCCTA',
  'CTTCCCTTCTCCTAT',
  'TTCCCTTCTCCTATC',
  'TCCCTTCTCCTATCC',
  'CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCC

# Step 2: • Cut reference bins into kmers = 15

In [14]:
# K-mers of reference
reference_kmers = (reference
  .select('sequence')
  .rdd
  .flatMap(lambda x: x)
  .map(extract_kmers))

In [15]:
reference_kmers.take(2)

[['CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCCTTGAA',
  'TCCTATCCCTTGAAA',
  'CCTATCCCTTGAAAA',
  'CTATCCCTTGAAAAA',
  'TATCCCTTGAAAAAT',
  'ATCCCTTGAAAAATT',
  'TCCCTTGAAAAATTG',
  'CCCTTGAAAAATTGT',
  'CCTTGAAAAATTGTC',
  'CTTGAAAAATTGTCA',
  'TTGAAAAATTGTCAT',
  'TGAAAAATTGTCATT',
  'GAAAAATTGTCATTT',
  'AAAAATTGTCATTTA',
  'AAAATTGTCATTTAT',
  'AAATTGTCATTTATT',
  'AATTGTCATTTATTT',
  'ATTGTCATTTATTTC',
  'TTGTCATTTATTTCT',
  'TGTCATTTATTTCTC',
  'GTCATTTATTTCTCT',
  'TCATTTATTTCTCTT',
  'CATTTATTTCTCTTA',
  'ATTTATTTCTCTTAT',
  'TTTATTTCTCTTATC',
  'TTATTTCTCTTATCC',
  'TATTTCTCTTATCCA',
  'ATTTCTCTTATCCAT',
  'TTTCTCTTATCCATA',
  'TTCTCTTATCCATAT',
  'TCTCTTATCCATATG',
  'CTCTTATCCATATGG',
  'TCTTATCCATATGGC',
  'CTTATCCATATGGCA',
  'TTATCCATATGGCAT',
  'TATCCATATGGCATA',
  'ATCCATATGGCATAA',
  'TCCATATGGCATAAT',
  'CCATATGGCATAATC',
  'CATATGGCATAATCA',
  'ATATGGCATAATCAA',
  'TATGGCATAATCAAA',
  'ATGGCATAAT

# Step 3: Collect all kmers and build a distinct kmer set

• Hint: you can use python “set” function

• Report the number of distinct kmers (N)

In [16]:
# Merge and distinct, collect to a set
kmers = set(reads_kmers 
  .union(reference_kmers)
  .flatMap(lambda x: x) 
  .distinct()
  .collect())

In [17]:
len(kmers)

72530

**The number of distinct kmers is 72,530**


In [22]:
reads

PythonRDD[34] at RDD at PythonRDD.scala:53

### Step4

In [115]:
from pyspark.ml.linalg import Vectors
import pandas as pd

In [162]:
def check_kmer_features(read):

  kmers_l = list(kmers)
  m = []
  

  for i in range(len(kmers)):
    if kmers_l[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (read,Vectors.sparse(len(kmers),m,v))



In [163]:
reads_features = reads.map(lambda x: check_kmer_features(x))

In [164]:
reads_features.take(1)

[('TCCTTACTGGTTTTGCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATC',
  SparseVector(72530, {172: 1.0, 850: 1.0, 1692: 1.0, 2458: 1.0, 4003: 1.0, 4611: 1.0, 6082: 1.0, 6575: 1.0, 7633: 1.0, 7926: 1.0, 9001: 1.0, 11636: 1.0, 13191: 1.0, 16387: 1.0, 17650: 1.0, 18186: 1.0, 18285: 1.0, 19256: 1.0, 19559: 1.0, 20326: 1.0, 20403: 1.0, 21798: 1.0, 22094: 1.0, 23410: 1.0, 23749: 1.0, 23953: 1.0, 23974: 1.0, 24733: 1.0, 25542: 1.0, 25959: 1.0, 26539: 1.0, 26778: 1.0, 27996: 1.0, 28213: 1.0, 30113: 1.0, 31480: 1.0, 34069: 1.0, 34680: 1.0, 35112: 1.0, 36107: 1.0, 37876: 1.0, 38474: 1.0, 38479: 1.0, 40068: 1.0, 40495: 1.0, 40783: 1.0, 41140: 1.0, 41709: 1.0, 42513: 1.0, 42585: 1.0, 42929: 1.0, 43083: 1.0, 43503: 1.0, 44230: 1.0, 45111: 1.0, 46254: 1.0, 50539: 1.0, 51937: 1.0, 53795: 1.0, 54047: 1.0, 54194: 1.0, 54919: 1.0, 55894: 1.0, 56222: 1.0, 58226: 1.0, 59433: 1.0, 59625: 1.0, 61056: 1.0, 61062: 1.0, 61217: 1.0, 61337: 1.0, 61469: 1.0, 61883: 1.0, 61972: 1

In [116]:
reads_feature_df = pd.DataFrame()

### Step5

In [165]:
reference_features = reference.select("sequence").rdd.map(lambda x:check_kmer_features(x['sequence']))

In [166]:
reference_features.take(1) 

[('CCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTAT',
  SparseVector(72530, {172: 1.0, 850: 1.0, 2586: 1.0, 4998: 1.0, 6082: 1.0, 7339: 1.0, 7633: 1.0, 8854: 1.0, 9001: 1.0, 9265: 1.0, 11636: 1.0, 13476: 1.0, 13838: 1.0, 14098: 1.0, 15257: 1.0, 15332: 1.0, 15838: 1.0, 16319: 1.0, 16669: 1.0, 17222: 1.0, 17650: 1.0, 18186: 1.0, 19256: 1.0, 19559: 1.0, 20326: 1.0, 20403: 1.0, 21798: 1.0, 22094: 1.0, 23024: 1.0, 25542: 1.0, 25636: 1.0, 26037: 1.0, 26539: 1.0, 27996: 1.0, 33472: 1.0, 33779: 1.0, 34069: 1.0, 34670: 1.0, 36146: 1.0, 38479: 1.0, 38744: 1.0, 40495: 1.0, 40783: 1.0, 41140: 1.0, 41709: 1.0, 42063: 1.0, 42513: 1.0, 42585: 1.0, 43083: 1.0, 43809: 1.0, 44230: 1.0, 45111: 1.0, 46254: 1.0, 46887: 1.0, 47328: 1.0, 48123: 1.0, 49417: 1.0, 49885: 1.0, 50177: 1.0, 51937: 1.0, 53096: 1.0, 53761: 1.0, 53795: 1.0, 54047: 1.0, 54095: 1.0, 57833: 1.0, 58226: 1.0, 58705: 1.0, 59433: 1.0, 61056: 1.0, 61217: 1.0, 61972: 1.0, 62467: 1.0, 62579: 